In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.isotonic import IsotonicRegression

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132495 entries, 0 to 1413024
Data columns (total 11 columns):
place_name_encoded       1132495 non-null int64
property_type_encoded    1132495 non-null int64
price_aprox_usd          1132495 non-null float64
superficie               1132495 non-null float64
Year                     1132495 non-null int64
Month                    1132495 non-null int64
seguridad                1132495 non-null bool
aire                     1132495 non-null bool
gimnasio                 1132495 non-null bool
cochera                  1132495 non-null bool
pileta                   1132495 non-null bool
dtypes: bool(5), float64(2), int64(4)
memory usage: 65.9 MB


# Isotonic Regression

In [5]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [6]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [7]:
iso = IsotonicRegression()
iso.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: iso.predict(x)[0],axis = 1)
precision = iso.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 66.47 % , error = 46257434434.250084


# Probamos cambiando las columnas y los datos

In [52]:
#Probamos varias combinaciones de columnas, la de mejor resultado fue esta

columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio','aire','pileta','cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [53]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [55]:
iso = IsotonicRegression()
iso.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: iso.predict(x)[0],axis = 1)
precision = iso.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 87.78 % , error = 16865005472.822279


# Calculamos los verdaderos datos

In [56]:
analizar = pd.read_csv('/home/mati/Desktop/properati_dataset_modificado.csv')

In [57]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: iso.predict(x)[0],axis = 1)

In [58]:
resultado = analizar.loc[:,['id','price_usd']]

In [59]:
resultado.to_csv('Isotonic_Regression_resultados.csv', index = False)